In [ ]:
!git clone https://github.com/saisritejakuppa/SignLangaugeRecognition.git

Cloning into 'SignLangaugeRecognition'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 0), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), 13.32 KiB | 4.44 MiB/s, done.


In [ ]:
!mkdir data
%cd /content/data
!wget https://www.sign-lang.uni-hamburg.de/meinedgs/videos/1583882/1583882.mp4
!wget https://www.sign-lang.uni-hamburg.de/meinedgs/openpose/1583882_openpose.json.gz

/content/data
--2023-01-27 17:31:40--  https://www.sign-lang.uni-hamburg.de/meinedgs/videos/1583882/1583882.mp4
Resolving www.sign-lang.uni-hamburg.de (www.sign-lang.uni-hamburg.de)... 134.100.55.40
Connecting to www.sign-lang.uni-hamburg.de (www.sign-lang.uni-hamburg.de)|134.100.55.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79408325 (76M) [video/mp4]
Saving to: ‘1583882.mp4’

1583882.mp4         100%[===================>]  75.73M  14.7MB/s    in 6.4s    

2023-01-27 17:31:48 (11.9 MB/s) - ‘1583882.mp4’ saved [79408325/79408325]

--2023-01-27 17:31:48--  https://www.sign-lang.uni-hamburg.de/meinedgs/openpose/1583882_openpose.json.gz
Resolving www.sign-lang.uni-hamburg.de (www.sign-lang.uni-hamburg.de)... 134.100.55.40
Connecting to www.sign-lang.uni-hamburg.de (www.sign-lang.uni-hamburg.de)|134.100.55.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232345769 (222M) [application/x-gzip]
Saving to: ‘1583882_openpose.json.gz’


In [ ]:
!gunzip /content/data/1583882_openpose.json.gz

In [ ]:
%cd /content
from SignLangaugeRecognition.scripts.keypointsgeneration.keypoints_utils import keypoints,SaveFrame
import numpy as np

# Path: main.py
openpose_json_path = '/content/data/1583882_openpose.json'
vidname = '/content/data/1583882.mp4'
imgspath = '/content/SignLangaugeRecognition/output/imgs'
heatmapspath = '/content/SignLangaugeRecognition/output/heatmaps'


process = keypoints(openpose_json_path)

data = process.ReadJson()


from tqdm import tqdm

count = 0
#make the dir imgs, heatmaps if not present
import os
if not os.path.exists(imgspath):
    os.makedirs(imgspath)

if not os.path.exists(heatmapspath):
    os.makedirs(heatmapspath)

for i in tqdm(range(0, 41864, 50)):
    points, dataset_good = process.KeyPointsExtract(0, data,i)

    if dataset_good:
        count += 1
        # print('The dataset is ', dataset_good)
        try:
            heatmaps = process.DrawHeatMaps(points)
            SaveFrame(vidname, i, f'{imgspath}/img_frame_'+str(i)+'.png')

            #save the heatmaps as npy file
            np.save(f'{heatmapspath}/heatmaps_'+str(i)+'.npy', heatmaps)

        except:
            print('The frame number is ', i)
            continue

print('The count is ', count)

/content


 16%|█▋        | 138/838 [00:40<05:27,  2.14it/s]

The frame number is  6850


100%|██████████| 838/838 [03:04<00:00,  4.53it/s]

The count is  194


In [ ]:
!pip install torchmetrics
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 26.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=6b4456365e7f5970dab37578bf2368d99a5aefeb3a54168230aa8fa1d442e65d
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully bui

In [ ]:
import sys
sys.path.append('/content/SignLangaugeRecognition')

from scripts.models.models import UNet
from scripts.utils.dataloader import ImageHeatmapDataset, showbatch
import torch
from torch.utils.data import DataLoader
from glob import glob
import torch.nn.functional as F
import torch.nn as nn
from torchmetrics import StructuralSimilarityIndexMeasure
from tqdm import tqdm
from torch.optim import Adam

#get early stopping
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
image_paths = glob('/content/SignLangaugeRecognition/output/imgs/*.png')
heatmap_paths = glob('/content/SignLangaugeRecognition/output/heatmaps/*.npy')

image_paths.sort()
heatmap_paths.sort()

dataset = ImageHeatmapDataset(image_paths, heatmap_paths)

# Set the split lengths
train_length = int(len(dataset) * 0.8)
val_length = int(len(dataset) * 0.15)
test_length = len(dataset) - train_length - val_length

# Use random_split to split the dataset
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_length, val_length, test_length])

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Loss(nn.Module):
    def __init__(self, vgg):
        super().__init__()
        self.mse = nn.MSELoss()
        self.ssim = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)
        self.vgg = vgg

    def forward(self, y_true, y_pred):
        # Mean squared error between the true and predicted images
        mse = self.mse(y_true, y_pred)

        # Structural similarity index (SSIM) between the true and predicted images
        ssim = 1 - self.ssim(y_pred, y_true)

        # Perceptual loss 
        # Extract features from the true and predicted images
        true_features = self.vgg(y_true)
        pred_features = self.vgg(y_pred)

        # Calculate the mean squared error between the true and predicted features
        pl = 0
        for true_feature, pred_feature in zip(true_features, pred_features):
            pl += self.mse(true_feature, pred_feature)

        # Combine the MSE, SSIM and perceptual loss into a single loss value
        loss = 0.4 * mse + 0.3 * ssim + 0.3*pl

        return loss, ssim, mse, pl

In [ ]:

from torchvision import models

model = UNet(22, 3)

#convert to device
model = model.to(device)

vgg = models.vgg19(pretrained=True).to(device)
loss_fn = Loss(vgg)
optimizer = Adam(model.parameters(), lr=1e-3)

# Initialize the scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
batch_size = 4

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def compute_val_loss(val_dataloader, model, loss_fn):
    with torch.no_grad():
        total_val_loss = 0
        for val_images, val_heatmaps in val_dataloader:
            val_images = val_images.to(torch.float).to(device)
            val_heatmaps = val_heatmaps.to(torch.float).to(device)
            val_generated_images = model(val_heatmaps)
        val_batch_loss,_,_,_ = loss_fn(val_images, val_generated_images)
        total_val_loss += val_batch_loss.item()
    return total_val_loss / len(val_dataloader)

In [ ]:
import wandb
wandb.init(project='SignGAN', entity='saisritejak')   

epoch,▁█▁█
loss,▁▁▁▁▁█▁▁▁▁▁▇▁▁▁▁▁▆▁▁▁▁▁▆▁▁
mse,█▇▇▆▄▄▃▆▇▆▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁▁▂▁▁▁▁▁▁
pl,█▇▇▆▄▄▃▆▇▆▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁▁▂▁▁▁▁▁▁
ssim,█▆▅▅▄▃▂▅▆▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▁▂▂▂▂▂▁▁▁▂▁
val_loss,█▃▁
epoch,1
loss,1.43055
mse,0.08577
pl,0.08577
ssim,0.39046


In [ ]:
# !mkdir models

In [ ]:
def train():
    # Iterate over the number of training epochs
    for epoch in range(num_epochs):
        # Iterate over the training data

        loss = 0
        cur_step = 0

        for images, heatmaps in tqdm(train_dataloader, desc="Epoch {}/{}".format(epoch+1, num_epochs)):
            # Move the data to the GPU (if available)
            images = images.to(torch.float).to(device)
            heatmaps = heatmaps.to(torch.float).to(device)

            # Forward pass
            generated_images = model(heatmaps)
            
            # Compute the loss
            batch_loss, ssim, mse, pl = loss_fn(images, generated_images)

            #log the ssim and mse
            wandb.log({'ssim': ssim})
            wandb.log({'mse': mse})
            wandb.log({'pl': pl})

            # Zero the gradients
            optimizer.zero_grad()

            # Backward pass
            batch_loss.backward()

            # Update the weights
            optimizer.step()

            # Add the loss to the total loss for the epoch
            loss += batch_loss.item()

            if cur_step % batch_log == 0:
                #log the loss
                wandb.log({'loss': batch_loss.item()})

                #log the images
                wandb.log({'images': [wandb.Image(images[0]), wandb.Image(generated_images[0])]})

            cur_step =  cur_step + 1

            # break
            
        # Print the current loss value
        print("Epoch: {}, Loss: {:.4f}".format(epoch+1, loss / len(train_dataloader)))

        #log the loss
        wandb.log({'loss': loss})

        #log the epoch
        wandb.log({'epoch': epoch})
        
        # Compute the validation loss
        val_loss = compute_val_loss(val_dataloader, model, loss_fn)
        wandb.log({'val_loss': val_loss})

        torch.save(model, f'models/unet_model_{epoch}.pt')

In [ ]:
num_epochs = 100
batch_log = 8
train()
torch.save(model, 'unet_model.pt')

Epoch 1/100: 100%|██████████| 39/39 [1:30:01<00:00, 138.50s/it]


Epoch: 1, Loss: 113816.5299


Epoch 2/100: 100%|██████████| 39/39 [1:31:27<00:00, 140.69s/it]


Epoch: 2, Loss: 0.1899


Epoch 3/100: 100%|██████████| 39/39 [1:32:58<00:00, 143.04s/it]


...

Epoch 5/100: 100%|██████████| 39/39 [1:31:31<00:00, 140.81s/it]


Epoch: 5, Loss: 0.1671


Epoch 6/100: 100%|██████████| 39/39 [1:31:11<00:00, 140.30s/it]


Epoch: 6, Loss: 0.1597


Epoch 7/100:  23%|██▎       | 9/39 [21:47<1:12:29, 144.97s/it]